In [ ]:
# 首先，步频，提特征
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import matplotlib.pyplot as plt
from tslearn.clustering import KShape
from tslearn.datasets import CachedDatasets
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
import numpy as np
import glob
from tslearn.generators import random_walks
import tslearn.metrics as metrics
os.chdir(r'Z:\main\Data Mining\College Experience Study Dataset')


def preprocessing(i,nickname,n,n1,n2):
    data = sensing_df[['uid','day',sensing_cols[i]]]
    data_main = data[np.isnan(data[sensing_cols[i]])]
    ind = list(data_main.index)
    data_main = data.drop(index=ind,axis=0)
    data_main= data_main[data_main[sensing_cols[i]] != 0]
    data_main = data_main.reset_index()
    data_main = data_main.drop(['index'], axis=1)
    mean_list = []
    uid = list(set(data_main['uid']))
    for x in uid:
        test = data_main[data_main['uid'] == x]
        mean_list.append(np.mean(list(test[sensing_cols[i]])))
    label = {'uid':uid,'mean':mean_list}
    label = pd.DataFrame(label)
    data_main = pd.merge(data_main,label,on='uid',how='outer')
    data_main[nickname] = data_main[sensing_cols[i]] - data_main['mean']
    data_main = data_main[['uid','day', nickname]]
    patient_name = list(set(list(data_main['uid'])))
    time_point = sorted(list(set(data_main['day'])))
    test = pd.DataFrame(columns=time_point,index=patient_name)
    test = test.fillna(0)
    for x in range(len(data_main)):
        test.loc[data_main.loc[x,'uid'],data_main.loc[x,'day']] = data_main.loc[x,nickname]
    test = np.array(test,dtype=np.float16)
    test = np.nan_to_num(test)
    np.random.shuffle(test)
    test = TimeSeriesScalerMeanVariance().fit_transform(test)
    sz = test.shape[1]
    seed = 0
    np.random.seed(seed)
    ks = KShape(n_clusters=5, verbose=True, random_state=seed)
    y_pred = ks.fit_predict(test)
    plt.figure(dpi=100,figsize=(20,20))
    for yi in range(n):
        plt.subplot(n1, n2, 1 + yi)
        for xx in test[y_pred == yi]:
            plt.plot(xx.ravel(), "k-", alpha=.2)
        # plt.plot(ks.cluster_centers_[yi].ravel(), "r-")
        plt.xlim(0, sz)
        plt.ylim(-6, 6)
        plt.title("Cluster %d" % (yi + 1))
        plt.tight_layout
    plt.show()

    plt.figure(dpi=100,figsize=(20,20))
    for yi in range(n):
        plt.subplot(n1, n2, 1 + yi)
        for xx in test[y_pred == yi]:
            plt.plot(xx.ravel(), "k-", alpha=.2)
        plt.plot(ks.cluster_centers_[yi].ravel(), "r-")
        plt.xlim(0, sz)
        plt.ylim(-6, 6)
        plt.title("Cluster %d" % (yi + 1))
        plt.tight_layout
    plt.show()
    label = {'uid' : patient_name,'sensing_cluster' : y_pred}
    label = pd.DataFrame(data=label)
    test = pd.merge(data_main, label, on=['uid'],how='outer')
    test = test[['uid','day',nickname,'sensing_cluster']]
    test.to_csv('final.'+ nickname +'.csv',index=False)
    return 

In [ ]:
merged_data_with_baseline = pd.read_csv('./tried/merged_data_with_baseline.csv')
merged_data_with_baseline = merged_data_with_baseline.drop(['step_ep_0','act_in_vehicle_ep_0','loc_dist_ep_0','stride_actual','phq4_score','stride_baseline'],axis=1)
merged_data_with_baseline

In [ ]:
stride = merged_data_with_baseline
patient_name = list(set(list(stride['uid'])))
time_point = sorted(list(set(stride['day'])))
test = pd.DataFrame(columns=time_point,index=patient_name)
test = test.fillna(0)
for x in range(len(stride)):
    test.loc[stride.loc[x,'uid'],stride.loc[x,'day']] = stride.loc[x,'stride_deviation']
test = np.array(test,dtype=np.float16)
test = np.nan_to_num(test)
np.random.shuffle(test)
test = TimeSeriesScalerMeanVariance().fit_transform(test)
sz = test.shape[1]
seed = 0
np.random.seed(seed)
ks = KShape(n_clusters=5, verbose=True, random_state=seed)
y_pred = ks.fit_predict(test)

In [ ]:
plt.figure(dpi=100,figsize=(20,20))
for yi in range(5):
    plt.subplot(4, 5, 1 + yi)
    for xx in test[y_pred == yi]:
        plt.plot(xx.ravel(), "k-", alpha=.2)
    # plt.plot(ks.cluster_centers_[yi].ravel(), "r-")
    plt.xlim(0, sz)
    plt.ylim(-6, 6)
    plt.title("Cluster %d" % (yi + 1))
    plt.tight_layout
plt.show()

plt.figure(dpi=100,figsize=(20,20))
for yi in range(5):
    plt.subplot(4, 5, 1 + yi)
    for xx in test[y_pred == yi]:
        plt.plot(xx.ravel(), "k-", alpha=.2)
    plt.plot(ks.cluster_centers_[yi].ravel(), "r-")
    plt.xlim(0, sz)
    plt.ylim(-6, 6)
    plt.title("Cluster %d" % (yi + 1))
    plt.tight_layout
plt.show()

In [ ]:
label = {'uid' : patient_name,'sensing_cluster' : y_pred}
label = pd.DataFrame(data=label)
test = pd.merge(merged_data_with_baseline, label, on=['uid'],how='outer')
test
test = test[['uid','day','stride_deviation','sensing_cluster']]
test.to_csv('final.strike.csv',index=False)

In [ ]:
sensing_cols = [
    'act_on_foot_ep_0',
    'act_walking_ep_0', 
    'audio_amp_mean_ep_0', 
    'audio_convo_duration_ep_0', 
    'audio_convo_num_ep_0', 
    'audio_voice_ep_0', 
    'loc_social_audio_amp', 
    'loc_social_audio_voice', 
    'loc_social_convo_duration', 
    'loc_social_convo_num', 
    'loc_social_dur', 
    'loc_social_still', 
    'loc_social_unlock_duration', 
    'loc_social_unlock_num',
    'sleep_duration', 
    'sleep_end', 
    'sleep_start'
]

In [ ]:
sensing_df = pd.read_csv(r'Z:\main\Data Mining\College Experience Study Dataset\Sensing\sensing.csv')

In [ ]:
preprocessing(0,'foot',5,4,5)


In [ ]:
preprocessing(1,'walking',5,4,5)

In [ ]:
preprocessing(2,'audio_amp',5,4,5)

In [ ]:
preprocessing(3,'convo_duration',5,4,5)

In [ ]:
preprocessing(4,'convo_number',5,4,5)

In [ ]:
preprocessing(5,'voice',5,4,5)

In [ ]:
preprocessing(6,'social_audio_amp',5,4,5)

In [ ]:
preprocessing(7,'social_audio_voice',5,4,5)

In [ ]:
preprocessing(8,'social_convo_duration',5,4,5)

In [ ]:
preprocessing(9,'social_convo_number',5,4,5)

In [ ]:
preprocessing(10,'social_duration',5,4,5)

In [ ]:
preprocessing(11,'social_still',5,4,5)

In [ ]:
preprocessing(12,'unlock_duration',5,4,5)

In [ ]:
preprocessing(13,'unlock_number',5,4,5)

In [ ]:
preprocessing(14,'sleep_duration',5,4,5)

In [ ]:
preprocessing(15,'sleep_start',5,4,5)

In [ ]:
preprocessing(16,'sleep_end',5,4,5)